In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base() 
Base.prepare(engine, reflect=True)
Table1 = Base.classes.Table1 
Table2 = Base.classes.Table2

session = Session(engine)
results1 = session.query(Table1.column1, Table1.column2).all()
results2 = session.query(Table2.column3, Table2.column4).all()

df1 = pd.DataFrame(results1, columns=["column1", "column2"])
df2 = pd.DataFrame(results2, columns=["column3", "column4"])
session.close()

# reflect the tables
plt.figure(figsize=(10,6))
plt.plot(df1["column1"], df1["column2"], label="Table1Data")
plt.plot(df2["column3"], df2["column4"], label="Table2Data")
plt.xlabel("X-Axis Label")
plt.ylabel("Y-Axis Label")
plt.title("Title of Plot")
plt.legend()
plt.show()

In [6]:
# View all of the classes that automap found
Base.classes.keys()

In [7]:
# Save references to each table
Table1 = Base.classes.Table1
Table2 = Base.classes.Table2

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.
Base= automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station 
session = Session(engine)
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
most_recent_date =pd.to_datetime(most_recent_date)

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
one_year_ago = most_recent_date - pd.DateOffset(years=1)
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

# Calculate the date one year from the last date in data set.
one_year_ago = most_recent_date - pd.DateOffset(years=1)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=["date", "prcp"])
df.set_index("date", inplace=True)

# Sort the dataframe by date
df.sort_index(ascending=True, inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(10,6))
df.plot(kind="bar", width=3)
plt.xlabel("Date")
plt.ylabel("Precipitation (inches")
plt.title("Precipitation in Last 12 Months")
plt.xticks(rotation=45)
plt.legend(["Precipitation"])
plt.tight_layout()
plt.show()


In [11]:
# Use Pandas to calcualte the summary statistics for the precipitation data
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
df = pd.read_sql("SELECT date, prcp FROM measurement", engine)
summary_stats = df["prcp"].describe()
print(summary_stats)

# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number stations in the dataset
total_stations_query = "SELECT COUNT(DISTINCT station) as total_stations FROM measurement"
total_stations_df = pd.read_sql(total_stations_query, engine)
print("Total number of stations:", total_stations_df['total_stations'].values[0])

In [13]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active_stations_query = "SELECT station, COUNT(*) as station_count FROM measurement GROUP BY station ORDER BY station_count DESC"
most_active_stations_df = pd.read_sql(most_active_stations_query, engine)
print("Most Active Stations:")
print(most_active_stations_df[['station', 'station_count']])
most_active_station_id = most_active_stations_df['station'][0]

In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temperature_query = f"SELECT MIN(tobs) as min_temperature, MAX(tobs) as max_temperaturem AVG(tobs) as avg_temperature FROM measurement WHERE station = '{most_active_station_id}'"
temperature_df = pd.read_sql(temperature_query, engine)
print("Temperature Statistics for Most Active Station:")
print(temperature_df)

In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last_12_months_query = f"SELECT date, tobs FROM measurement WHERE station = '{most_active_station_id}' AND date <= DATE('now', '-1 year')"
last_12_months_df = pd.read_sql(last_12_months_query, engine)

plt.figure(figsize=(10, 6))
plt.hist(last_12_months_df['tobs'], bins=12)
plt.xlabel("Temperature (F)")
plt.ylabel("Frequency")
plt.title("Temperature Observation Data for Most Active Station (Last 12 Months)")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()